# Calculate PDF profile

This notebook illustrates how to calculate $G(r)$ in EasyDiffraction using `pdffit2` library (J. Phys, Condens. Matter **19**, 335219 (2007). doi: https://doi.org/10.1088/0953-8984/19/33/3352190).  
The result of the calculation is qualitatively compared with experimental X-ray data.

In [ ]:
import os

from easyCore.Fitting.Fitting import Fitter
from easyDiffractionLib import Phases
from easyDiffractionLib.interface import InterfaceFactory as Calculator
from easyDiffractionLib.Profiles.P1D import PDFParameters
from easyDiffractionLib.Jobs import Powder1DCW
from easyDiffractionLib.Interfaces.pdffit2 import readGRData

# for plotting
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
output_notebook()
FIGURE_WIDTH = 900
FIGURE_HEIGHT = 300

## Load reduced experimental data

`Ni-xray.gr` contains reduced X-ray data with Ni sample. Its structure is a a header with metadata and then an array of 4 columns: $r$, $G(r)$, and the error related to these 2 quantities.  
The reduction was done with `pdfgetx2`.


In [ ]:
# data_fname = os.path.realpath('examples\\PDF\\Ni-xray.gr')
data_fname = os.path.realpath('Ni-xray.gr')
data = readGRData(data_fname)

## Load crystallographic reference data

In [ ]:
cif_fname = os.path.realpath('Ni.cif')
phases = Phases.from_cif_file(cif_fname)
phases

In [ ]:
phases.phase_names

We get the information about the phase from the reference cif file:

In [ ]:
phases.get_parameters()

## Define G(r)

- Select `pdffit2` as Calculator
- Create a "continuous 1D powder"-type job named `Ni_simulated` for Ni sample using the phases from the loaded cif and `PDF` parameters (set below)
- Associate this job and the calculator

In [ ]:
calculator = Calculator()
calculator.switch("Pdffit2")

In [ ]:
parameters = PDFParameters()

job = Powder1DCW('Ni_simulated', 
                 parameters=parameters, 
                 phases=phases, 
                 interface=calculator)

fitter = Fitter(job, 
                calculator.fit_func)

### Set job parameters 

In [ ]:
parameters = job.parameters

List of available parameters with their default values, units, status (i.e., to be refined or fixed) and possible constraints

In [ ]:
parameters.get_parameters()

Here is a short description for some of them:
- Q_damp: PDF Gaussian dampening envelope due to limited $Q$-resolution. The Gaussian envelope is of the form $\exp(-\frac{(r Q_{damp})^2}{2})$
- Q_broad: PDF peak broadening from increased intensity noise at high $Q$
- delta1: coefficient for $(1/r)$ contribution to the peak sharpening.
- delta2: coefficient for $(1/r^2)$ contribution to the peak sharpening.

Using these notations, the PDF peak width is expressed as 

$$ \sigma_{ij} \sqrt{1-\frac{\delta_1}{r_{ij}}-\frac{\delta_2}{r_{ij}^2}+Q_{broad}^2 r_{ij}^2},$$ where $i$, $j$ correspond to two different atoms.  

It contains contributions from thermal and zero point displacements and static disorder. $\sigma_{ij}'$ is the peak width without correlation.   
The first two terms correct for the effects of correlated motion. The term $\delta_2/r^2$ describes the low temperature behavior, and $\delta_1/r$ describes the high temperature case. Since the two parameters are highly correlated, one will in practice choose which one to refine.  

The last term models the PDF peak broadening as a result of
the Q-resolution of the diffractometer. In many cases this term will only be significant for refinements over wider r -ranges. Note that the Q resolution also results in an exponentialdampening of the PDF peaks which is modeled using the parameter Q_damp.


Specify $Q_{max}$, $Q_{damp}$, $Q_{broad}$ and wavelength values used to calculate PDF. 

In [ ]:
parameters.qmax = 70 #30
parameters.qdamp = 0.03
parameters.qbroad = 0.0 #0.024
parameters.wavelength = 0.126514
parameters.delta2 = 1
#parameters.delta1 = 0.9
parameters.spdiameter = 0.0

In [ ]:
# Check update of set parameter values
parameters.get_parameters()

### Set pattern parameters 
#### Patterns related to all phases

In [ ]:
global_patterns = job.pattern

In [ ]:
global_patterns.get_parameters()

In [ ]:
global_patterns.zero_shift = 0.16

#### Patterns related to phase 1

In [ ]:
phase1_patterns = job.phases[0]

In [ ]:
phase1_patterns.get_parameters()

In [ ]:
# Modify the scale for pattern 1
job.phases[0].scale = 1

In [ ]:
# Check modified value
phase1_patterns.scale

### Select r from experimental data and create the simulated G(r)

In [ ]:
x_data = data[:, 0]

y_data = job.create_simulation(x_data)

## Plot

In [ ]:
Gobs = data[:, 1]
Gfit = y_data

Gdiff = Gobs - Gfit
Gdiff_baseline = -15  # Offset difference curve for plotting

Gdiff_show = Gdiff + Gdiff_baseline

fig = figure()
fig.xaxis.axis_label = 'r (Å)'
fig.yaxis.axis_label = r"$$G (Å^{-2})\$$"
fig.title.text = 'Fit of nickel to x-ray experimental PDF'

fig.scatter(x_data, Gobs, legend_label='G(r) Data', fill_alpha=0., line_color='steelblue', marker='circle')
fig.line(x_data, Gfit, legend_label='G(r) Fit', color='orangered', line_width=2)
fig.line(x_data, Gdiff_show, legend_label='G(r) Diff', color='grey', line_width=2)
show(fig)

Note that the experimental and calculated curves can be quite different. But the aim of this notebook was to detail how to simulate a 1D PDF pattern in EasyDiffraction using `pdffit2` library.  
Please refer to `Fitting_PDF_Profile.ipynb` in order to see how to fot this type of curve.